<a href="https://colab.research.google.com/github/pratap977/CS6910/blob/main/CNN_updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install split-folders

In [2]:
!pip install wandb

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from PIL import Image as im
import math as mh
from matplotlib import pyplot
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Activation,Dense,Flatten
from keras.preprocessing.image import load_img as im
from keras.preprocessing.image import save_img as sim
# import torch

In [4]:
import os
from random import shuffle
import numpy as np
import splitfolders
from PIL import Image
from torchvision import transforms
import numpy

In [5]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras import layers,models
import wandb

In [6]:
src_url = "https://storage.googleapis.com/wandb_datasets/nature_12K.zip"
src_zip = "nature_12K.zip"

In [7]:
%%capture
!curl -SL $src_url > $src_zip
!unzip $src_zip

In [8]:
data_folders="/content/inaturalist_12K/train"

In [9]:
splitfolders.ratio(data_folders, output="outputfin", seed=1337, ratio=(.9, .1), group_prefix=None)

Copying files: 9999 files [00:11, 855.67 files/s]


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
!rm -rf /content/drive/MyDrive/outputfin

In [13]:
!cp -r /content/outputfin /content/drive/MyDrive/

In [ ]:
!rm -rf nature_12K.zip

In [ ]:
!rm -rf inaturalist_12K/train

In [6]:
class_labels = os.listdir("/content/drive/MyDrive/outputfin/train")

In [7]:
lab={l:i for l,i in zip(class_labels,range(10))}
lab

{'Amphibia': 9,
 'Animalia': 0,
 'Arachnida': 5,
 'Aves': 2,
 'Fungi': 3,
 'Insecta': 8,
 'Mammalia': 4,
 'Mollusca': 6,
 'Plantae': 1,
 'Reptilia': 7}

In [8]:
data_augmentation  = transforms.Compose([
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

In [15]:
imgs_tr = []
filenames = []

truth_tr = []
for l in class_labels:
  if l.startswith("."):
    continue
  imgs_per_class = os.listdir(os.path.join("/content/drive/MyDrive/outputfin/train", l))
  for img in imgs_per_class:
    
    filenames.append(img.split(".")[0])
    
    img_path = os.path.join("/content/drive/MyDrive/outputfin/train", l, img)
    # img = image.load_img(img_path, target_size=(224, 224))
    # img = image.img_to_array(img)
    img=Image.open(img_path)
    img = img.resize((224,224))
    if img.mode == 'L':
                continue
    normalized_image = data_augmentation(img)
    truth_tr.append(lab[l])
    # don't forget to rescale test images to match the range of inputs
    # to the network
    # img = img/255.0
    imgs_tr.append(numpy.asarray(normalized_image).transpose(1,2,0))
x_train=np.asarray(imgs_tr)
y_train=np.asarray(truth_tr)
del imgs_tr
del truth_tr

In [22]:
x_train.shape

(8997, 224, 224, 3)

In [16]:
imgs_val = []
filenames = []

truth_val = []
for l in class_labels:
  if l.startswith("."):
    continue
  imgs_per_class = os.listdir(os.path.join("/content/drive/MyDrive/outputfin/val", l))
  for img in imgs_per_class:
    
    filenames.append(img.split(".")[0])
    
    img_path = os.path.join("/content/drive/MyDrive/outputfin/val", l, img)
    # img = image.load_img(img_path, target_size=(299, 299))
    # img = image.img_to_array(img)
    img=Image.open(img_path)
    img = img.resize((224,224))
    if img.mode == 'L':
                continue
    normalized_image = data_augmentation(img)
    truth_val.append(lab[l])
    # don't forget to rescale test images to match the range of inputs
    # to the network
    # img = img/255.0
    imgs_val.append(numpy.asarray(normalized_image).transpose(1,2,0))
x_val=np.asarray(imgs_val)
y_val=np.asarray(truth_val)
del imgs_val
del truth_val

In [21]:
x_val.shape

(999, 224, 224, 3)

In [ ]:
imgs_val = []
filenames = []

truth_val = []
for l in class_labels:
  if l.startswith("."):
    continue
  imgs_per_class = os.listdir(os.path.join("/content/drive/MyDrive/inaturalist_12K/val", l))
  for img in imgs_per_class:
    
    filenames.append(img.split(".")[0])
    truth_val.append(lab[l])
    img_path = os.path.join("/content/drive/MyDrive/inaturalist_12K/val", l, img)
    img = image.load_img(img_path, target_size=(224, 224))
    img = image.img_to_array(img)
    # don't forget to rescale test images to match the range of inputs
    # to the network
    img = img/255.0
    imgs_val.append(img)
x_test=np.asarray(imgs_val)
y_test=np.asarray(truth_val)
del imgs_val
del truth_val

In [10]:
import pathlib

In [11]:
train_data = tf.keras.preprocessing.image_dataset_from_directory(
                      directory = pathlib.Path("/content/drive/MyDrive/inaturalist_12K/train"),
                      labels = 'inferred',  
                      label_mode = 'categorical',
                      color_mode = 'rgb',
                      batch_size = 32,
                      image_size = (224, 224),
                      shuffle = True,
                      seed = 17,
                      validation_split = 0.1,
                      subset = 'training')

val_data = tf.keras.preprocessing.image_dataset_from_directory(
                      directory = pathlib.Path("/content/drive/MyDrive/inaturalist_12K/train"),
                      labels = 'inferred',  
                      label_mode = 'categorical',
                      color_mode = 'rgb',
                      image_size = (224, 224),
                      shuffle = True,
                      seed = 17,
                      validation_split = 0.1,
                      subset = 'validation')

Found 9999 files belonging to 10 classes.
Using 9000 files for training.
Found 9999 files belonging to 10 classes.
Using 999 files for validation.


In [20]:
x_test.shape

(2000, 224, 224, 3)

In [ ]:
sweep_config = {
                  'method': 'grid',
                  'metric':{
                            'name':'accuracy',
                            'goal':'maximize'
                            },
                  'parameters': {
                                'filsize'   : {'values': [32, 64, 96, 128, 256]},
                                'kersize'   :{'values':[3,5,7,9]},
                                'dropout'   :{'values':[0.1,0.3,0.4]}, 
                                'denseunits':{'values':[256,100,50,60]}, 
                                'epochs'    :{'values':[5,10,20,30]},                          
                              }
              }

In [30]:
sweep_config = {
                  'method'    : "random",
                  'metric':{
                            'name':'val_accuracy',
                            'goal':'maximize'
                            },
                  'parameters': {
                                  'flt'                 : {'values' :[32, 64,16,]  },
                                  'filterorg'               : {'values' :[1,2,0.5] },
                                  'kerneldim'               : {'values' :[3,4,5,7]  },
                                  'activation'              : {'values' :['sigmoid','relu','tanh','elu'] },
                                  'neurons'                 : {'values' :[20,32,64,128,512] },
                                  'optimizer'               : {'values' :['adam'] },
                                  'lr'                      : {'values' :[1e-3,1e-4,2e-3,3e-3,4e-3,3e-4] },
                                  'epochs'                  : {'values' :[15,20,30] },
                                  'batchsize'               : {'values' :[16,32,64,128] },
                                  'dropout'                 : {'values' :[0.2,0.25,0.3,0,0.1,0.5] },
                                  'batchnormalisation'       : {'values' :['YES','NO']} ,
                                }
                }

In [ ]:
sweep_config

{'method': 'grid',
 'metric': {'goal': 'maximize', 'name': 'accuracy'},
 'parameters': {'denseunits': {'values': [256, 100, 50, 60]},
  'dropout': {'values': [0.1, 0.3, 0.4]},
  'epochs': {'values': [5, 10, 20, 30]},
  'filsize': {'values': [32, 64, 96, 128, 256]},
  'kersize': {'values': [3, 5, 7, 9]}}}

In [12]:
def create_CNN(fil,ker,activ,batchnormalisation,dropout,neurons):
  cnn=models.Sequential()
  cnn.add(layers.Conv2D(filters=fil[0], kernel_size=(ker[0], ker[0]), activation='relu',input_shape=(224, 224, 3)))
  cnn.add(layers.MaxPooling2D((2, 2)))
  if(batchnormalisation=='YES'):
    cnn.add(layers.BatchNormalization())
        
        
  cnn.add(layers.Conv2D(filters=fil[1], kernel_size=(ker[1], ker[1]), activation='relu'))
  cnn.add(layers.MaxPooling2D((2, 2)))
  if(batchnormalisation=='YES'):
    cnn.add(layers.BatchNormalization())
                                                  
  cnn.add(layers.Conv2D(filters=fil[2], kernel_size=(ker[2], ker[2]), activation='relu'))
  cnn.add(layers.MaxPooling2D((2, 2)))
  if(batchnormalisation=='YES'):
    cnn.add(layers.BatchNormalization())
         
  cnn.add(layers.Conv2D(filters=fil[3], kernel_size=(ker[3], ker[3]), activation='relu'))
  cnn.add(layers.MaxPooling2D((2, 2)))
  if(batchnormalisation=='YES'):
    cnn.add(layers.BatchNormalization())

  cnn.add(layers.Conv2D(filters=fil[4], kernel_size=(ker[4], ker[4]), activation='relu'))
  cnn.add(layers.MaxPooling2D((2, 2)))
  
  if(batchnormalisation=='YES'):
    cnn.add(layers.BatchNormalization())


  cnn.add(layers.Flatten())
  if(dropout!=0):
    cnn.add(layers.Dropout(dropout))
  cnn.add(layers.Dense(neurons, activation=activ))
  
  cnn.add(layers.Dense(10, activation='softmax'))
  return cnn

In [ ]:
activation: relu
wandb: 	batchnormalisation: YES
wandb: 	batchsize: 128
wandb: 	dropout: 0.5
wandb: 	epochs: 5
wandb: 	filterorg: 0.5
wandb: 	flt: 16
wandb: 	kerneldim: 5
wandb: 	lr: 0.003
wandb: 	neurons: 20
wandb: 	optimizer: rmsprop

In [62]:
cnn=create_CNN([16,8,4,2,1],[5,5,5,5,5],'relu','YES',0.5,20)

In [ ]:
wandb: 	activation: relu
wandb: 	batchnormalisation: NO
wandb: 	batchsize: 16
wandb: 	dropout: 0.1
wandb: 	epochs: 5
wandb: 	filterorg: 2
wandb: 	flt: 32
wandb: 	kerneldim: 5
wandb: 	lr: 0.0003
wandb: 	neurons: 20
wandb: 	optimizer: rmsprop

In [18]:
cnn=create_CNN([32,64,128,256,512],[5,5,5,5,5],'relu','YES',0.2,20)

In [19]:
cnn.compile(optimizer= tf.keras.optimizers.RMSprop(learning_rate=0.003),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
# cnn.fit(train_data, batch_size=128,epochs=10,validation_data=val_data)
cnn.fit(x_train,y_train, batch_size=128,epochs=10,validation_data=(x_val,y_val))

Epoch 1/10
71/71 [==============================] - 15s 195ms/step - loss: 4.2349 - accuracy: 0.1237 - val_loss: 12.2529 - val_accuracy: 0.1101
Epoch 2/10
71/71 [==============================] - 13s 179ms/step - loss: 2.3942 - accuracy: 0.1226 - val_loss: 3.2206 - val_accuracy: 0.1251
Epoch 3/10
71/71 [==============================] - 13s 178ms/step - loss: 2.2995 - accuracy: 0.1370 - val_loss: 2.6466 - val_accuracy: 0.1241
Epoch 4/10
71/71 [==============================] - 13s 181ms/step - loss: 2.2510 - accuracy: 0.1497 - val_loss: 2.5528 - val_accuracy: 0.1241
Epoch 5/10
71/71 [==============================] - 13s 180ms/step - loss: 2.2577 - accuracy: 0.1478 - val_loss: 2.3353 - val_accuracy: 0.1411
Epoch 6/10
71/71 [==============================] - 13s 177ms/step - loss: 2.2235 - accuracy: 0.1609 - val_loss: 2.2390 - val_accuracy: 0.1431
Epoch 7/10
71/71 [==============================] - 14s 193ms/step - loss: 2.2131 - accuracy: 0.1645 - val_loss: 2.2766 - val_accuracy: 0.151

In [28]:
def conlay():
  wandb.init(config=sweep_config, magic=True)  
  f= wandb.config.flt 
  fo=  wandb.config.filterorg                           
  fil=[ (int)(f*(fo)**i) if((int)(f*(fo)**i)>0) else  1 for i in range(5)]
  kd=wandb.config.kerneldim
  ker=[kd for i in range(5)]
  if(wandb.config.activation=="sigmoid"):
    activ=tf.keras.activations.sigmoid
  elif(wandb.config.activation=="tanh"):
    activ=tf.keras.activations.tanh
  elif(wandb.config.activation=="relu"):
    activ='relu'
  else:
    activ=tf.keras.activations.elu
  if(wandb.config.optimizer=="sgd"):
    optim = tf.keras.optimizers.SGD(learning_rate=wandb.config.lr)
  elif(wandb.config.optimizer=="adam"):
    optim = tf.keras.optimizers.Adam(learning_rate=wandb.config.lr)
  elif(wandb.config.optimizer=="momentum"):
    optim = tf.keras.optimizers.SGD(learning_rate=wandb.config.lr,momentum=wandb.config.gama)
  elif(wandb.config.optimizer=="rmsprop"):
    optim = tf.keras.optimizers.RMSprop(learning_rate=wandb.config.lr)
      
  cnn =create_CNN(fil,ker,activ,wandb.config.batchnormalisation,wandb.config.dropout,wandb.config.neurons)             
  cnn.compile(optimizer=optim,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  cnn.fit(x_train, y_train, batch_size=wandb.config.batchsize,epochs=wandb.config.epochs,validation_data=(x_val,y_val),callbacks=[wandb.keras.WandbCallback()])
  del cnn


In [31]:
wandb.login()


wandb: Currently logged in as: pratap49 (use `wandb login --relogin` to force relogin)


True

In [33]:
sweep_id = wandb.sweep(sweep_config,project='A2-part-A1')
wandb.agent(sweep_id,conlay,project='A2-part-A1')

wandb: Agent Starting Run: bc9tc6p7 with config:
wandb: 	activation: tanh
wandb: 	batchnormalisation: YES
wandb: 	batchsize: 32
wandb: 	dropout: 0.1
wandb: 	epochs: 20
wandb: 	filterorg: 0.5
wandb: 	flt: 32
wandb: 	kerneldim: 4
wandb: 	lr: 0.0001
wandb: 	neurons: 512
wandb: 	optimizer: adam


Create sweep with ID: uww8d6oe
Sweep URL: https://wandb.ai/pratap49/A2-part-A1/sweeps/uww8d6oe


Epoch 1/20
282/282 [==============================] - 13s 43ms/step - loss: 2.2623 - accuracy: 0.1596 - val_loss: 2.2353 - val_accuracy: 0.1542 - _timestamp: 1648845671.0000 - _runtime: 22.0000
Epoch 2/20
282/282 [==============================] - 12s 41ms/step - loss: 2.1400 - accuracy: 0.2193 - val_loss: 2.1063 - val_accuracy: 0.2372 - _timestamp: 1648845682.0000 - _runtime: 33.0000
Epoch 3/20
282/282 [==============================] - 29s 103ms/step - loss: 2.0965 - accuracy: 0.2367 - val_loss: 2.0785 - val_accuracy: 0.2372 - _timestamp: 1648845711.0000 - _runtime: 62.0000
Epoch 4/20
282/282 [==============================] - 14s 48ms/step - loss: 2.0716 - accuracy: 0.2481 - val_loss: 2.0708 - val_accuracy: 0.2412 - _timestamp: 1648845725.0000 - _runtime: 76.0000
Epoch 5/20
282/282 [==============================] - 13s 45ms/step - loss: 2.0470 - accuracy: 0.2579 - val_loss: 2.0583 - val_accuracy: 0.2503 - _timestamp: 1648845738.0000 - _runtime: 89.0000
Epoch 6/20
282/282 [=========

accuracy,▁▃▄▅▅▆▆▆▆▆▇▇▇▇▇▇████
epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
loss,█▆▅▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
val_accuracy,▁▇▇▇█▆█▇██▇▇▇▇▇█▆██▇
val_loss,█▃▃▂▂▃▂▁▁▂▂▁▂▂▁▂▂▁▁▂
accuracy,0.32833
best_epoch,7
best_val_loss,2.0346
epoch,19
loss,1.88751
val_accuracy,0.24124


wandb: Agent Starting Run: xt2ibl2t with config:
wandb: 	activation: sigmoid
wandb: 	batchnormalisation: YES
wandb: 	batchsize: 16
wandb: 	dropout: 0.25
wandb: 	epochs: 15
wandb: 	filterorg: 1
wandb: 	flt: 64
wandb: 	kerneldim: 3
wandb: 	lr: 0.0001
wandb: 	neurons: 512
wandb: 	optimizer: adam


...

wandb: Agent Starting Run: xn8hejh3 with config:
wandb: 	activation: sigmoid
wandb: 	batchnormalisation: YES
wandb: 	batchsize: 16
wandb: 	dropout: 0
wandb: 	epochs: 20
wandb: 	filterorg: 0.5
wandb: 	flt: 16
wandb: 	kerneldim: 4
wandb: 	lr: 0.003
wandb: 	neurons: 32
wandb: 	optimizer: adam


Run xn8hejh3 errored: InternalError()
wandb: ERROR Run xn8hejh3 errored: InternalError()
wandb: Agent Starting Run: 901snmht with config:
wandb: 	activation: elu
wandb: 	batchnormalisation: NO
wandb: 	batchsize: 32
wandb: 	dropout: 0.1
wandb: 	epochs: 20
wandb: 	filterorg: 2
wandb: 	flt: 32
wandb: 	kerneldim: 7
wandb: 	lr: 0.002
wandb: 	neurons: 128
wandb: 	optimizer: adam


Run 901snmht errored: InternalError()
wandb: ERROR Run 901snmht errored: InternalError()
wandb: Agent Starting Run: 1q2c488k with config:
wandb: 	activation: sigmoid
wandb: 	batchnormalisation: YES
wandb: 	batchsize: 16
wandb: 	dropout: 0.25
wandb: 	epochs: 30
wandb: 	filterorg: 2
wandb: 	flt: 64
wandb: 	kerneldim: 5
wandb: 	lr: 0.001
wandb: 	neurons: 20
wandb: 	optimizer: adam


Run 1q2c488k errored: InternalError()
wandb: ERROR Run 1q2c488k errored: InternalError()
wandb: Agent Starting Run: p13gz0wt with config:
wandb: 	activation: sigmoid
wandb: 	batchnormalisation: YES
wandb: 	batchsize: 32
wandb: 	dropout: 0.2
wandb: 	epochs: 30
wandb: 	filterorg: 0.5
wandb: 	flt: 64
wandb: 	kerneldim: 5
wandb: 	lr: 0.001
wandb: 	neurons: 20
wandb: 	optimizer: adam


Run p13gz0wt errored: InternalError()
wandb: ERROR Run p13gz0wt errored: InternalError()


In [52]:
def conlayss():
  # wandb.init(config=sweep_config, magic=True)    
  cnn = models.Sequential([
                     
    layers.Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D((2,2)),
    
    layers.Conv2D(64,  kernel_size=(3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64, kernel_size=(3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64,  kernel_size=(3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64,  kernel_size=(3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dropout(0.2),

    
    layers.Dense(30, activation='relu'),
    layers.Dense(10, activation='softmax')
  ])
  #gc.collect()
  # heyGPU=torch.cuda.is_available()
  # if heyGPU: cnnModel=cnnModel.cuda()
  print("start")
  train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
  train_dataset = train_dataset.shuffle(buffer_size=1024).batch(64)

    # Prepare the validation dataset.
  val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
  val_dataset = val_dataset.batch(64)
  print("end")
  cnn.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  cnn.fit(train_dataset,val_dataset,batch_size=64 ,epochs=10)
  del cnn
  # torch.cuda.empty_cache()

In [ ]:
y_train[5]

0

In [53]:
conlayss()

start


InternalError: ignored